### <font face="微软雅黑" color=#A52A2A > 1.为什么Bagging降方差，boosting降偏差

偏差指的是算法的期望预测与真实值之间的偏差程度，反映了模型本身的拟合能力。  
方差度量了同等大小的训练集的变动导致学习性能的变化，刻画了数据扰动所导致的影响。  

集成模型其实方差和偏差都有照顾，这里提到的其实是两种集成模型增加轮数达到的效果。  

当模型越复杂时，拟合能力就越好，模型的偏差就越好。但此时如果换一组数据可能模型的变化就会很大，即模型方差变大，所以复杂的模型容易造成过拟合；当模型简单的时候，即使换一组数据，得出的学习器分类效果与之前分类器的效果也不会很大，即模型方差很小，但由于模型过于简单，导致偏差会很大。  

对于Bagging算法来说，由于我们并行的训练很多的分类器的目的就是降低这个方差，因为采用了相互独立的基分类器的数量多了，h的值就会靠近。所以对于每个基分类器的目的就是如何降低这个偏差，所以我们会采用深度很深并且不剪枝的决策树。

对于Boosting来说，每一步我们都会在上一轮的基础上更加的拟合原数据，所以可以保证偏差。所以对于每个基分类器来说，问题就是如何选择方差更小的分类器，即更简单的弱分类器，所以我们选择深度很浅的决策树。

### <font face="微软雅黑" color=#A52A2A >2.随机森林与GBDT区别

随机森林是bagging的一个扩展变体。RF在以决策树为基学习器构建bagging集成的基础上，引入了随机属性的选择。传统的决策树在选择划分属性的时候是在当前节点的属性集合中选择一个最优属性；而在RF中，对基学习器的每个节点，先从该节点的属性集合中随机选择一个包含k个属性的子集，再从子集中选择一个最优属性进行划分。如果k=d，那么基学习器和传统决策树相同；如果k=1，那么就是随机选择属性进行划分。RF的基学习器的多样性不仅来自样本扰动，而且来自属性扰动，最终泛化性能可以通过个体学习器的差异增加得到进一步提升。

GBDT是一个boosting的算法，每一轮迭代都会生成一个决策树模型，最后的预测结果是把所有的弱学习器的结果相加。如果预测值看成自变量，那么boosting的过程就是损失函数在做梯度下降的过程，每一轮迭代但是在拟合伪残差。

### <font face="微软雅黑" color=#A52A2A > 3. 为什么xgboost效果不如随机森林

xgboost是boosting的过程，迭代过程主要是降低偏差，让预测更接近真实值。不如RF的可能原因有很多，如果xgboost训练的轮数不够，或者是其他一些超参数的调节不优，就会影响最后的结果。如果在调整的比较好的情况下，又是用的同样一组训练集，那么更可能的原因是xgboost产生了overfitting，产生了比较大的方差，导致在训练集的误差很小，但是在测试集上的误差比较大。

对于RF而言，由于它是基于bagging的集成算法，它会在使用重采样的方式生成多个训练集，然后训练多个学习器，最后取平均作为结果。另外，RF在选择分裂的属性的时候也加入了随机性，这样的方式更robust，所以能够比较好的降低方差。

综上所述，xgboost不如随机森林的原因可能是训练集和测试集的分布出现了不一样的特点，导致boosting方法出现了泛化误差比较大的情况。

### <font face="微软雅黑" color=#A52A2A > 4. xgb对gbdt有什么提升，lgb对xgb有什么提升，为什么有了xgb还要使用lgb

xgboost是高阶版的GBDT，它们之间主要有着以下不同：  
原理上：  
1. 传统GBDT主要采用了cart作为基学习器，而xgboost还可以支持线性分类器，这个时候xgboost相当于带L1和L2正则化项的logistic回归（分类问题）或者线性回归（回归问题）。
2. GBDT采用了一阶展开，而xgboost采用了泰勒二阶展开，二阶信息能让梯度收敛更快。 
3. xgboost加入了正则项，用于控制模型的复杂度，降低过拟合风险。正则项惩罚了叶子节点数量和叶子节点的输出值的平方和。
4. xgboost在计算分裂节点属性的时候，可以使用分桶算法，加快计算速度。同时也支持针对二阶导数的加权分桶。
5. xgboost加入了缺失值处理。

系统上：
1. 并行的优化。不同维度特征排序可以同时进行，不同叶子节点的分裂可以同时进行。
2. cache的hit miss优化
3. 使用硬盘处理超大数据集。数据压缩之后再放到内存，用计算换IO时间；数据sharding到多个磁盘，读取时同时使用。

LightGBM对于xgboost的提升主要体现在对于大数据集的处理上。xgboost在面对大数据集的时候，会出现计算速度慢的缺点。因为在选择分裂属性的时候，如果是精确求解，需要扫描每一个属性的每一个取值；即便是使用的近似解法，特征数量很多的时候需要扫描每一个特征。  
所以，LightGBM的在样本维度和特征维度都做了优化。GOSS的采样可以大幅减少样本数，尽量保持梯度绝对值大的样本，同时保持数据分布不变。EFB把互斥特征绑定，减少需要扫描的特征数。

### <font face="微软雅黑" color=#A52A2A > 5. GBDT是如何做分裂的

GBDT只是boosting的一种形式，里面的弱学习器是可以选择多种的，只要满足低方差、高偏差的即可（因为GBDT主要是通过多轮迭代，降低偏差来降低泛化误差）。一般而言选择cart树比较多，所以GBDT如何分裂其实是在问cart如何做分裂的。如果是回归树，那么是用平方差最小来选择特征；如果是分类树，是用gini系数。

### <font face="微软雅黑" color=#A52A2A > 6. GBDT是如何做分类任务

GBDT不管是做分类还是回归任务，里面的基学习器都只能是回归树，因为迭代过程是需要针对残差相加减的，而类别的相加减是没有意义的。在做分类任务的时候，针对每一个可能的类别，都需要训练一棵树。比如在三分类问题中，每一轮我们都会训练三棵树，如果样本的label是第二类别，那么三棵树的理论输出应该是[0,1,0]。第一轮训练三棵树之后，会加上softmax函数，求得三个类别对应的概率，然后用[0,1,0]减去softmax处理之后的概率得到残差，作为第二轮的训练lable。

### <font face="微软雅黑" color=#A52A2A > 7. GBDT是如何产生特征组合

参考Facebook LR+GBDT，待完善

### <font face="微软雅黑" color=#A52A2A > 6. xgboost为什么要二阶展开

1. 二阶信息本身就能让梯度收敛更快更准确，这在牛顿法优化里面已经证明。可以认为一阶导指引梯度方向，二阶导指引梯度方向如何变化。其实xgboost的迭代过程就是牛顿法的过程。牛顿法是用来求解f(x)=0的根的方法，通过不断地迭代，来逼近零点。而f(x)最优化的过程就是求f'(x)=0的过程，我们把f'(x)看成函数，使用牛顿法，就有x(n+1) = x(n)-f'(x)/f"(x)。而-f'(x)/f"(x)就是每一轮树的叶子节点输出。
2. xgboost使用二阶展开可以统一形式，只要损失函数是二次可微的，能够给出g和h，那么就可以进行计算，这样会方便自定义损失函数。

### <font face="微软雅黑" color=#A52A2A > 7. 牛顿法与SGD区别

牛顿法是用来求解f(x)=0的根的方法，我们把泰勒公式展开到二阶，对其求导，然后令f'(x)=0，得到的表达式x(n+1)=x(n)-f'(x)/f"(x)。这能让梯度收敛更快更准确。而SGD只是指明了梯度方向。

### <font face="微软雅黑" color=#A52A2A > 8. 牛顿法能用于非凸函数吗

不能，梯度下降和牛顿法停止的条件都是f'(x)=0。梯度下降方法使用的是负梯度方向，这个方向对应的值是下降的，它有可能会被trap在一个local minima，但是它不会往高处走；但是牛顿法不一样，它变化的方向是-f'(x)/f"(x)，这里的f"(x)是不定的，它有可能会使函数的迭代走到极大值方向。

### <font face="微软雅黑" color=#A52A2A > 9. ID3, C4.5, CART是什么，有什么优势

ID3节点分裂的属性选择用的是信息增益，即分裂前的信息熵减去分裂后的信息熵，选差值最大的作为分裂特征。信息熵的形式是sum(-plogp),信息熵越小表示纯度越高。但是信息增益有一个缺点，就是会倾向于选择取值很多的属性，比如编号这种的，因为这样分类之后每个节点就一个样本，当然纯度很高，但是没有意义。所以就引申出了C4.5。C4.5使用的是信息增益比，就是信息增益再除上一个该属性的熵（每个取值样本占比是p），这样就能消除影响。CART树在分类任务中使用的是gini指数。Gini(D)=sum(1-p^2),代表着从数据集中取出两个样本它们不相同的概率。和熵相似，越小纯度越高。而gini_index（D,A）等于划分之后各节点gini(Di)乘上占比权重再求和。最后的属性选择使gini_index最小的属性。

C4.5是ID3的加强版，它相比ID3，1.可以处理缺失值；2.可以接受类别型和数值型；3.可以剪枝，防止overfitting

cart树都可以，分类问题中使用的是基尼指数，回归问题使用的是平方误差

### <font face="微软雅黑" color=#A52A2A > 10. 决策树是如何剪枝的

分为预剪枝和后剪枝两种：预剪枝是指在划分前后计算在测试集上的精度，如果下降就不分裂，但是容易造成欠拟合；后剪枝是指在构建完整颗树之后，自底而上考察所有非叶子节点，看是否能剪枝，能否剪枝的条件也是在测试集上精度是否有提升。

还可以通过设置最大叶子节点个数进行预剪枝；熵减少的数量小于阈值；树的深度

### <font face="微软雅黑" color=#A52A2A > 11. 树模型有什么优势

简单便捷，便于部署，可解释性强

### <font face="微软雅黑" color=#A52A2A > 12. overfitting的原因及处理方式，xgboost中有哪些参数可以使用

过拟合原因：
1. 训练集太少
2. 训练集和测试集特征分布不一致
3. 模型过于复杂，记住了训练集中不具有代表性的特征

处理方式：
1. 增加训练数据
2. 对gbdt，可以降低训练轮数，限制叶子节点数量，限制树的深度，每个节点最少的样本量
3. 正则化
4. dropout
5. early stop (交叉验证)
6. 增加随机性，在特征和样本的维度上随机采样

xgboost中主要有两大类降低方式降低过拟合
1. 控制模型复杂度，包括max_depth(最大树深度), min_child_weight(每个节点最小的二次导数和), min_split_loss(最小分裂收益), reg_lambda, reg_alpha(正则项)
2. 增加随机性，robust to noise，包括subsample，colsample，也可以降低学习率，增大轮数

### <font face="微软雅黑" color=#A52A2A > 13. 分类模型如何处理数据不平衡问题，xgboost中有哪些参数可以使用

1. 对多数的类别进行欠采样，结果可能导致信息损失
2. 对少数的类别进行过采样，结果可能导致重复使用数据，造成过拟合
3. 对数据进行采用的过程中通过相似性同时生成并插样“少数类别数据”，叫做SMOTE算法
4. 阈值调整（threshold moving），将原本默认为0.5的阈值调整到 较少类别/（较少类别+较多类别）即可
5. 结合集成学习来有效的使用数据，假设正例数据n，而反例数据m个。我们可以通过欠采样，随机无重复的生成（k=n/m）个反例子集，并将每个子集都与相同正例数据合并生成k个新的训练样本。我们在k个训练样本上分别训练一个分类器，最终将k个分类器的结果结合起来，比如求平均值。

评价模型时使用AUC，不要用精度

xgboost可以使用的参数有scale_pos_weight, 多类别可以使用multiclass_weight

### <font face="微软雅黑" color=#A52A2A > 14. xgboost如何处理缺失值，lightgbm差加速和直方图算法

xgboost允许缺失值的存在
1. 如果是训练样本的缺失值，类别型的数据直接作为一类；对于连续型的缺失值，分别处理将missing该特征值的样本分配到左叶子结点和右叶子结点的两种情形，计算增益后选择增益大的方向进行分裂即可，记录下缺失值对应的是左子树还是右子树，在预测的时候作为默认分支。
2. 如果是训练样本没有缺失值，但是在预测样本中有，则默认右子树作为分支。

直方图算法基本思想是把离散的浮点特征离散化成k个整数，对每一个叶子节点的每一个特征，都会构建一个直方图，并且统计每个桶对应的统计值（一阶导数和样本量）。有了统计量之后，就不再需要presort特征。直接遍历所有桶对应的整数值，找到最佳分裂点即可。

### <font face="微软雅黑" color=#A52A2A > 15. AUC的含义及计算方式

AUC指的是ROC曲线下与坐标轴围成的面积，纵坐标是真正例率(真的正例率)，即真实标签为1，预测也为1的概率；横坐标是假正例率(假的正例率)，即真实标签为0，但是预测为1的概率。我们肯定是希望纵坐标越大越好，横坐标倾向于0。如果是直接把(0,0)和(1,1)两个点连接，这个ROC曲线就是完全随机预测，不管真实样本的标签是什么，预测为1的概率相同。

AUC曲线评估的其实是分类器对样本的正确排序能力。大多数情况下，分类器的工作原理是给出一个预测值和一个阈值，如果预测值大于这个阈值，我们认为是正例，反之则是负例。我们把样本按输出值大小排序，阈值从上往下移动的过程就是ROC曲线形成的过程，而AUC，就是评估这个排序好不好的指标。排序越好，意味着正例都排在前面，ROC曲线在纵坐标上会最快地趋近1，而后再往右，这就是最好的情况，AUC=1。  
  
AUC的含义就是任取两个样本，正例排在负例前面的概率。如果正例全部排在了负例前面，那么AUC=1。 AUC = P(P+ > P-)。

AUC的计算方式：
1. 最直接的计算方式，计算ROC下面的面积。把样本排序好之后，从上到下扫描，如果真实样本是1，那么ROC曲线往上移动1/m；如果真实样本是0，那么ROC曲线往右移动1/m。
2. 统计一下所有的 M×N(M为正类样本的数目，N为负类样本的数目)个正负样本对中，有多少个组中的正样本的score大于负样本的score。想象极端的情况，所有的正例都排在了最后，那么 sumRank-(M*(M+1)/2.0) 就等于0

### <font face="微软雅黑" color=#A52A2A > 16. AUC与PRC的异同点，适用的场景

1. AUC可以帮助选择分类器选择最佳阈值，最靠近左上角的点是分类误差最小的。
2. ROC曲线很容易可以评价出两个分类器哪个更好

AUC的计算方法同时考虑了分类器对于正例和负例的分类能力，在测试样本不平衡的情况下，依然能够对分类器作出合理的评价。  
比如在反欺诈的场景中，如果欺诈是正例，那么正例的比例很小(假设为1%)。如果把所有样本预测为负，用准确率评估，那么accuracy=99%。如果全部预测为正，则查准率和查全率都很高，但这个分类器明显不好。

但是如果用AUC，纵坐标为0(因为没有预测为正的样本)，最后连接(0,0)和(1,1)，得到AUC=0.5。也就是说在测试样本不平衡的时候，依然反映了分类器的真实性能。 或者可以这么思考，即便是在样本不均衡的情况下，ROC也能衡量排序的好坏，如果一个分类器比较好，正例都排在负例前面，那么roc是正常的随着阈值往下，在纵轴上升很快；如果分类器不好，负例随机地穿插在正例之间，那么ROC曲线也会和随机的(0,0)和(1,1)连接线接近，所以这和样本分布没有关系。

### <font face="微软雅黑" color=#A52A2A > 17. L1和L2正则的对比

L1是权重绝对值和的形式；L2是特征权重系数的平方和形式。

为什么L1会得到稀疏解，即最后大量特征的稀疏会被置为0，而系数不为0的是我们需要的特征（可作为特征筛选）。  
我们可以在二维空间中分析：在二维空间中，平方和的形式呈现出来的是一个圆，为绝对值和的形式呈现的是一个菱形。对l1来说，在求最小值的时候，最可能的交点就是在菱形的顶点。

### <font face="微软雅黑" color=#A52A2A > 18. GD, SGD, Mini-Batch GD

Gradient Descent(或者叫Batch Gradient Descent)，批量梯度下降法是最原始的形式，它是指在每一次迭代时使用所有样本来进行梯度的更新。  
Stochastic Gradient Descent，随机梯度下降法不同于批量梯度下降，随机梯度下降是每次迭代使用一个样本来对参数进行更新。  
Mini-Batch Gradient Descent，小批量梯度下降，是对批量梯度下降以及随机梯度下降的一个折中办法。其思想是：每次迭代使用batch_size个样本来对参数进行更新。

### <font face="微软雅黑" color=#A52A2A > 19. 小样本预测方法

### <font face="微软雅黑" color=#A52A2A > 20. 特征筛选有哪些方法

1. 缺失值占比
2. 方差情况（方差太小的可以删去）
3. 两个特征的相关性，如果两个特征相关性很强，意味着信息重复(共线性，偏相关分析)
4. Multicolinearity
5. PCA
6. Cluster Analysis
7. Correlation with target(如chi2,F检验，皮尔逊相关系数）
8. Forward selection（循环，不断加入当前最好的特征）
9. backward selection（循环，不断减去一个特征）
10. stepwise selection（循环加入特征，一定步之后删去一个特征）
11. lasso（利用l1正则来筛选特征）
12. tree based
13. 信息增益

信息增益筛选特征:  
其实就是和决策树筛选的原理一样，看按照这个特征分裂之后，熵增加了多少，选择熵增加最多的k个特征，就实现了特征筛选。

皮尔逊相关系数：  


### <font face="微软雅黑" color=#A52A2A > 21. 数据预处理的方法

数据标准化：(feature scaling的四种方法)  
1. rescaling (x-min)/(max-min)
2. mean normalization (x-mean(x))/(max-min)
3. Standardization(Z-score normalization)  (x-mean(x))/std(x)
4. Scaling to unit length  x/abs(x)



为什么要对连续数据离散化：  
1. 离散特征的增加和减少都很容易，易于模型的快速迭代（离散型0-1直接就看权重是否有效；而连续型需要重新训练）
2. 离散化后的特征对异常数据有很强的鲁棒性（>30岁）
3. 稀疏向量内积乘法运算速度快，计算结果方便存储，容易扩展
4. 逻辑回归属于广义线性模型，表达能力受限；单变量离散化为N个后，每个变量有单独的权重，相当于为模型引入了非线性，能够提升模型表达能力，加大拟合
5. 离散化后可以进行特征交叉，由M+N个变量变为M*N个变量，进一步引入非线性，提升表达能力
6. 特征离散化后，模型会更稳定，降低过拟合
7. 特征离散化以后，起到了简化了逻辑回归模型的作用，降低了模型过拟合的风险。




数据离散化方法：  

有监督：  
卡方分箱（卡方检验是计算X^2的值，值越大越相关，即偏离理论不相关的程度；这里先按连续值属性排序，把所有连续值看成独立的区间，然后不断计算相邻区间和y的卡方值，选取最小的进行合并区间。合并的区间计算的卡方值必须小于一定阈值才能合并，这个阈值根据实际情况选取。）  
最小熵法分箱（分箱让整体熵最小）  

无监督：  
等距，等频率

### <font face="微软雅黑" color=#A52A2A > 22. 离线训练AUC提升了，但是在线上并没有，是什么原因

1. 发生了数据泄露，比如离线训练的时候使用了用户数据，但是在时间上的划分没有注意
2. 线上线下使用的特征不一样，比如离线训练使用的特征都是每天处理得到的，但是线上的特征来不及处理，只用了原来的老特征，所以造成不一致
3. 数据分布不一致，线上的数据线下没有见过，“冰山”下的数据排序没有做好

### <font face="微软雅黑" color=#A52A2A > 23. 加入了新的特征后，指标反而变差了，这个时候应该怎么做